In [1]:
import pandas as pd
from metasynth import MetaDataset
import wget
from pathlib import Path
import json

In [2]:
dtypes = {
    "Survived": "category",
    "Pclass": "category",
    "Name": "string",
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category"
}

In [3]:
titanic_fp = Path("titanic.csv")
if not titanic_fp.is_file():
    wget.download("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

In [4]:
df = pd.read_csv(titanic_fp, dtype=dtypes)

In [5]:
dataset = MetaDataset.from_dataframe(df)

In [6]:
df["Name"].dropna().values

<StringArray>
[                            'Braund, Mr. Owen Harris',
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
                              'Heikkinen, Miss. Laina',
        'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
                            'Allen, Mr. William Henry',
                                    'Moran, Mr. James',
                             'McCarthy, Mr. Timothy J',
                      'Palsson, Master. Gosta Leonard',
   'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
                 'Nasser, Mrs. Nicholas (Adele Achem)',
 ...
                                  'Markun, Mr. Johann',
                        'Dahlberg, Miss. Gerda Ulrika',
                       'Banfield, Mr. Frederick James',
                              'Sutehall, Mr. Henry Jr',
                'Rice, Mrs. William (Margaret Norton)',
                               'Montvila, Rev. Juozas',
                        'Graham, Miss. Margaret Edith',
            'Johnston, Miss. 

In [7]:
dataset.synthesize(1000)["Name"].dropna().values

<StringArray>
[   'DqbyckotiiM|',    'FkazatyjkvRK',             'QKa', 'KwxwkcdiqkaaiMr',
            'Cc,U',         'IbhhbD ',  'SvdnhbtszbtmGa', 'PqwrlufjpksumE#',
  'ZzdpxaxznmebNW',  'Lzkyzihgmaly 7',
 ...
  "Nnbucatkozwe'r",  'OhvkuwwbkhdnNh',         'PmcbtD,', 'RdpnoghzrmxtxK4',
  'BkedqffkgohG\r',           'UwjM;',          'Vvlq,^',   'HkhaobyqqkyN{',
   'LzgmqdtqodmMr',     'Kvwbxuwvu-!']
Length: 1000, dtype: string

In [8]:
dataset.to_json("test.json")

In [9]:
print(MetaDataset.from_json("test.json"))

AttributeError: 'NoneType' object has no attribute 'groups'

In [ ]:
dataset.to_xml("test.xml")

In [ ]:
dataset.to_dict()

In [ ]:
print(MetaDataset.from_xml("test.xml"))

In [ ]:
dataset.to_dict()